In [4]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [21]:
ANALYSIS_PROMPT = "You are a research assistant interested in political capture in India. Right now, you are researching whether \
politicians with agricultural assets are more prone to taking actions that favor agricultural businesses. Your first task is to assess whether \
a politician owns agricultural assets. To do this, you will be given a table that summarizes their own immovable assets, as well as those owned by \
their spouse and dependents. Summarize this information by mentioning who owns assets and their valuation"

USER_TEMPLATE = "Name: {name}\nElectionYear: {election_year}\nImmovable Assets: {assets}"

In [22]:
import pandas as pd

In [23]:
winners_details_folder = "../data/winner_details"
winners = pd.read_csv("../data/winners.csv")

In [24]:
win_sample = winners.sample(1)
name = win_sample["name"].values[0]
unique_id = win_sample["unique_id"].values[0]
election_year = unique_id.split("_")[0]
immov_assets = pd.read_csv(
        os.path.join(winners_details_folder, unique_id, "immovable_assets.csv")
).to_string()

messages = [
        {"role": "system", "content": ANALYSIS_PROMPT},
        {"role": "user", "content": USER_TEMPLATE.format(
                name=name,
                election_year=election_year,
                assets=immov_assets
        )}
]
response = openai.chat.completions.create(
        messages=messages,
        model="gpt-4o-mini"
)

In [25]:
response.choices[0].message.content

'Shalley Chaudhary, a politician from Haryana, has declared ownership of various immovable agricultural assets as part of her affidavit for the 2024 elections. The following summarizes the ownership and valuation of these assets:\n\n### Summary of Immovable Assets\n\n**Self-Owned Assets**\n1. **Agricultural Land**:\n   - **Location**: Villages Bhupani and Atmadpur, Faridabad\n   - **Total Area**: 5 acres in Bhupani, 6 Karnal in Atmadpur\n   - **Purchase Dates**: Various, with the earliest being June 8, 1995.\n   - **Total Valuation**: ₹15,50,00,000 (15 Crore+)\n\n**Spouse-Owned Assets**\n1. **Agricultural Land**:\n   - Various parcels in Ambala and Mohali with a shared interest.\n   - Total Valuation: ₹12,50,00,000 (12 Crore+)\n2. **Non-Agricultural Land**: \n   - Locations in Faridabad.\n   - Total Valuation: ₹28,00,00,000 (28 Crore+)\n\n**Dependents**: No immovable assets reported.\n\n### Total Current Market Value\n- **Self**: ₹37,50,00,000 (37 Crore+)\n- **Spouse**: ₹32,50,00,000 (

In [ ]:
def get_llm_response_from_row(row):
        name = row["name"].values[0]
        unique_id = row["unique_id"].values[0]
        immov_assets = pd.read_csv(
                os.path.join(winners_details_folder, unique_id, "immovable_assets.csv")
        )
        

,Unnamed: 0,name,link,state_year,unique_id
3488,3488,Rajesh Tripathi,https://myneta.info/uttarpradesh2022/candidate...,uttarpradesh2022,uttarpradesh2022_3825


In [ ]:
openai.chat.completions.create()